# **Reporte sobre la Extracción, Transformación y Carga: steam_games, user_reviews y user_items**

En este informe, exploraremos el proceso de Extracción, Transformación y Carga (ETL) de datos relacionados con los juegos disponibles en la plataforma **Steam**. Estos datos contienen información variada, como género, desarrolladores, precio, entre otros aspectos relevantes.

Nuestro objetivo principal es preparar estos datos para un análisis efectivo, garantizando su idoneidad y consistencia. Para iniciar, importaremos las bibliotecas necesarias. Es fundamental contar con estas bibliotecas instaladas previamente para asegurar una ejecución sin problemas.

A lo largo de este informe, nos concentraremos en identificar y resolver posibles problemas en los datos, aplicar técnicas de limpieza y preprocesamiento, y finalmente, almacenar los datos transformados para futuros análisis y exploraciones.

## Requisitos

🔎Asegúrate de instalar las siguientes bibliotecas antes de ejecutar el código

* pandas
* numpy
* pyarrow
* gdown
* langdetect
* nltk


Para instalar estas bibliotecas debes abrir una terminal o ventana de línea de comandos y ejecutar el siguiente comando:

<span style="background-color: #f2f2f2; color: black;">pip install numpy pandas gdown langdetect nltk pyarrow</span>


In [1]:
# Importamos la librería pandas para el análisis y manipulación de datos tabulares
import pandas as pd

# Importamos la librería numpy para realizar operaciones matemáticas y numéricas eficientes
import numpy as np

# Importamos la librería gdown para descargar archivos de Google Drive de manera sencilla
import gdown

# Langdetect es una biblioteca para detectar automáticamente el idioma en el que está escrito un texto
from langdetect import detect

# NLTK (Natural Language Toolkit) es una plataforma para construir programas Python para trabajar con datos de lenguaje humano
import nltk

# SentimentIntensityAnalyzer es una herramienta en NLTK para análisis de sentimientos
from nltk.sentiment import SentimentIntensityAnalyzer

# Importamos la librería json para trabajar con datos en formato JSON
import json

# Importamos la librería os para interactuar con el sistema operativo, por ejemplo, para manejar archivos y directorios
import os

#pyarrow es necesaria para crear los archivos PARQUET

import warnings
warnings.filterwarnings("ignore")

A continuacion, realizaremos las etapas del porceso de ETL:


# 1. Descargar el conjunto de datos original desde la fuente especificada:

**Fuente de datos: output_steam_games.json**

El archivo original está en formato JSON, donde los valores que podrían ser cadenas de caracteres, números, arreglos o valores booleanos no están entre comillas dobles. A pesar de esto, gracias a la flexibilidad de la librería Pandas para manejar diversos formatos de datos, se puede interpretar el contenido del JSON correctamente. A continuación, se realizará la descarga automática del archivo (se recomienda esta opción).

De cualquier manera, te dejare el link a la carpeta de Google drive: https://drive.google.com/drive/folders/1Heu44S33UQG2ls5Gij7QYsi52z7Qfw9D?usp=drive_link

In [2]:
def descargar_leer_json(Id, nombre_archivo):
    """
    Descarga un archivo JSON con el Id proporcionado, lo guarda localmente
    con el nombre especificado y lo lee en un DataFrame de pandas.

    Parameters:
    - Id (str): Id del archivo en Google Drive.
    - nombre_archivo (str): Nombre del archivo.

    Returns:
    - df (pd.DataFrame): DataFrame de pandas con los datos del archivo JSON.
    """

    # Verificar si el archivo ya existe localmente (Asegúrese que sea la versión reparada)
    if not os.path.exists(nombre_archivo):
        # Si no existe, descargar el archivo desde Google Drive
        gdown.download(id=Id, output=nombre_archivo, quiet=False)

    # Lectura del JSON en un DataFrame de pandas
    df=pd.read_json(nombre_archivo, encoding='utf-8')

    return df

In [50]:
#id del archivo en Google Drive y nombre del archivo local
Id = "1sU1jJcnmoFOYqqADz5y8Knpro6eYtXWd"
output = 'output_steam_games.json'

#Utilizar la función (descargar_leer_json) para descargar y leer el JSON
df_Games = descargar_leer_json(Id, output)

# 2. Análisis e interpretación de los datos
Nos sumergimos en el conjunto de datos, explorando su estructura, revisando las primeras filas, obteniendo una visión general y analizando las estadísticas descriptivas para comprender su contenido.

In [51]:
# Mostrar las filas del dataframe
df_Games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


En este primer vistazo, se observa una gran cantidad de datos de tipo 'None' al inicio del dataframe.
Esto podria indicar una deficiencia en la fuente de estos datos o en la forma en la que se cargaron.
Se deben revisar estas instancias del proceso para garantizar futuros analisis mas completos y detallados.
Tambien se observan datos estructurados o anidados en algunas columnas ('genre',  'tags' y 'specs').

Veamos un poco de la informacion general del dataframe y su estructura:

In [52]:
df_Games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


Con esta informacion mas detallada de su estructura podemos sacar varias conclusiones:

* Las columnas 'app_name', 'url', 'reviews_url, 'early_acces' e 'id' tienen todos los valores no nulos.
* La columna 'release_date' tiene datos de fechas y se considera castearla al tipo datetime.
* 'genres', 'tags' y 'specs' contienen listas de categorias.
* Se considera convertir la columna 'early_access' al tipo bool.
* Por ultimo la columna 'price', se considera castearla al tipo float y donde el sean freetoplay dejarlos en 0. 

# 3. Limpieza y procesamiento

Abordaremos valores nulos y duplicados.
Detectaremos y eliminaremos columnas irrelevantes, si es pertinente

In [53]:
# Eliminar filas con todos los valores nulos
df_Games = df_Games.dropna(how='all').reset_index(drop=True)
df_Games.shape

(32135, 13)

In [54]:
# Reemplazar valores vacios, 'null' y 'None' por NaN en todas las columnas
df_Games.replace(['', 'null', 'None'], np.nan, inplace=True)

In [55]:
# Renombrar columnas
df_Games.rename(columns={'app_name': 'name', 'id': 'item_id'}, inplace=True)

In [56]:
# Cantidad de valores nulos en las columnas 'publisher' y 'developer'
df_Games[['publisher', 'developer']].isnull().sum()

publisher    8061
developer    3299
dtype: int64

In [57]:
# Verificar si 'developer' está vacía y 'publisher' está llena, luego copiar el valor
df_Games.loc[df_Games['developer'].isnull() & ~df_Games['publisher'].isnull(), 'developer'] = df_Games['publisher']

In [58]:
# Eliminar registros donde item_id es nulo o NaN
df_Games = df_Games.dropna(subset=['item_id'])

In [59]:
# Casteamos la columna release_date a datetime y exrtaemos el año (solo eso nos interesa)
df_Games['release_date']=pd.to_datetime(df_Games['release_date'], errors='coerce', exact=False)
df_Games['release_year'] = df_Games['release_date'].dt.year.astype('Int64')

# Descartamos la columna 'release_date'
df_Games = df_Games.drop(columns=['release_date'])

In [73]:
# Función para convertir precios a tipo float y ajustar los "freetoplay" a 0
def convert_price(price):
    if price is None:
        return 0.0
    elif isinstance(price, str) and 'free' in price.lower():
        return 0.0
    else:
        try:
            return float(price)
        except ValueError:
            return 0.0
# Aplicar la función a la columna 'price'
df_Games['price'] = df_Games['price'].apply(lambda x: convert_price(x))


In [75]:
# Convertimos la columna 'genres' a tipo lista, conservando NaN
df_Games['genres'] = df_Games['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Expandimos las listas en filas
df_genres = df_Games.explode('genres')

# Obtener la lista de géneros únicos, incluyendo los valores nulos
lista_generos = df_genres['genres'].unique()

df_Games['tags'] = df_Games['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_Games['genres'] = df_Games.apply(
    lambda row: [tag for tag in row['tags'] if tag in lista_generos] if isinstance(row['tags'], list) else row['genres'],
    axis=1
    )

In [76]:
# Se eliminan las columnas 'title','url','reviews_url','early_access','publisher', 'tags', 'specs'
# por considerarse irrelevantes
df_Games.drop(['title','url','reviews_url','early_access','publisher','tags','specs'], axis=1, inplace=True)

In [77]:
df_Games

,genres,name,price,item_id,developer,release_year
0,"[Strategy, Action, Indie, Casual, Simulation]",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
1,"[Free to Play, Strategy, Indie, RPG]",Ironbound,0.00,643980.0,Secret Level SRL,2018
2,"[Free to Play, Simulation, Sports, Casual, Indie]",Real Pool 3D - Poolians,0.00,670290.0,Poolians.com,2017
3,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400.0,彼岸领域,2017
4,"[Action, Indie, Casual, Sports]",Log Challenge,2.99,773570.0,None,<NA>
...,...,...,...,...,...,...
32130,"[Strategy, Indie, Casual, Simulation]",Colony On Mars,1.99,773640.0,"Nikita ""Ghost_RUS""",2018
32131,"[Strategy, Indie, Casual]",LOGistICAL: South Africa,4.99,733530.0,Sacada,2018
32132,"[Indie, Simulation, Racing]",Russian Roads,1.99,610660.0,Laush Dmitriy Sergeevich,2018
32133,"[Indie, Casual]",EXIT 2 - Directions,4.99,658870.0,"xropi,stev3ns",2017


# 4. Almacenar el conjunto de datos procesado
Guardamos los datos procesados en un nuevo archivo en formato PARQUET en la carpeta 'archivos_procesados' para facilitar su acceso y reutilización posterior.
Puedes guardarlo en el formto que mas te guste, si asi lo deseas. Personalmente elijo PARQUET para este proyecto por el gran volumen de datos que traen estos archivos, es mas eficiente en rcursos de la pc.

In [78]:
df_Games.to_parquet('C:\\Users\\Juampi\\Desktop\\Henry\\PI_MLOps\\Datasets\\archivos_procesados\\steam_games_cleaned.parquet')

Terminamos con ouput_steam_games.json, repetiremos este proceso con los 2 archivos restantes: australian_users_items.json y australian_user_review.json

# 1. Descargar el conjunto de datos:

**Fuente de datos: australian_users_items.json**

In [ ]:
#id del archivo en Google Drive y nombre del archivo local
Id = "1-p1ygUl2OqYK2uEd-wRmIVXUZ_J6wGVi"
output = 'australian_users_items.json'

#Utilizar la función (descargar_leer_json) para descargar y leer el JSON
df_usersItems = descargar_leer_json(Id, output)

# 2. Analisis e interpretacion de los datos

In [ ]:
df_respaldo = df_usersItems

In [ ]:
df_usersItems = df_respaldo

In [ ]:
#Echemos un vistazo
df_usersItems

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982480,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066624,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700688,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759264,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548336,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [ ]:
#Veamos su estructura
df_usersItems.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  int64 
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.0+ MB


Observamos que el dataframe no tiene archivos nulos, es un buen indicador de que su metodo de recoleccion
y posterior carga fueron procesos limpios.
Ademas se observa que la columna 'items' tiene estructura de lista, veamos un poco mas.

In [ ]:
primer_diccionario = df_usersItems['items'].iloc[0][0]

print("Primer diccionario de la primer fila:")
print(primer_diccionario)

Primer diccionario de la primer fila:
{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}


La columna 'items' se presenta de forma anidada, es decir, como una lista de diccionarios. En este proceso de normalización, desglosamos dicha columna para obtener una nueva columna por cada clave presente en los diccionarios, asegurando al mismo tiempo que se conserve la relación con las columnas 'steam_id', 'items_count', 'user_id' y 'user_url'.

# 3. Limpieza y procesamiento

In [ ]:
# Desanidamos la columna 'items'

desanidado = []
for index, row in df_usersItems.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    user_items_count = row['items_count']
    for df_items_dict in row['items']:
        df_items_dict['user_id'] = user_id
        df_items_dict['user_url'] = user_url
        df_items_dict['items_count'] = user_items_count
        desanidado.append(df_items_dict)

df_usersItems = pd.DataFrame(desanidado)

In [ ]:
df_usersItems

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,user_url,items_count
0,10,Counter-Strike,6,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
1,20,Team Fortress Classic,0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
2,30,Day of Defeat,7,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
3,40,Deathmatch Classic,0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
4,50,Half-Life: Opposing Force,0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,277
...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7
5153205,373330,All Is Dust,0,0,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,http://steamcommunity.com/profiles/76561198329...,7


In [ ]:
# Convertimos la columna 'playtime_forever' de minutos a horas
df_usersItems['hours_game'] = df_usersItems['playtime_forever'] / 60

# Redondeamos los valores a dos decimales (opcional)
df_usersItems['hours_game'] = df_usersItems['hours_game'].round(2)

In [ ]:
# Obtener los 5 valores máximos
top_5_max = df_usersItems.nlargest(5, 'hours_game')

# Obtener los 5 valores mínimos
top_5_min = df_usersItems.nsmallest(5, 'hours_game')

# Mostrar los resultados
print("Top 5 valores máximos:")
print(top_5_max[['hours_game']])

print("\nTop 5 valores mínimos:")
print(top_5_min[['hours_game']])

Top 5 valores máximos:
         hours_game
587715     10712.88
2499068    10588.25
4075729    10540.87
1495340    10223.52
1836985    10001.13

Top 5 valores mínimos:
   hours_game
1         0.0
3         0.0
4         0.0
5         0.0
6         0.0


In [ ]:
# Contar las filas con valor 0 en la columna 'hours_game'
filas_con_cero = (df_usersItems['hours_game'] == 0).sum()

# Imprimir el resultado
print(f"Existen {filas_con_cero} filas con valor 0 en la columna 'hours_game'.")

Existen 1867963 filas con valor 0 en la columna 'hours_game'.


In [ ]:
# Eliminar las filas con valor 0 en la columna 'hours_game'
df_usersItems = df_usersItems[df_usersItems['hours_game'] != 0]

Se identificaron 1,847,730 registros con un valor de 0 en la columna 'hours_game'. Dado que el análisis de horas jugadas es un requisito, se ha tomado la decisión de eliminar todos los registros con este valor nulo en la columna 'hours_game', ya que no contribuyen al estudio propuesto.

In [ ]:
#Eliminamos columnas no relevantes
df_usersItems = df_usersItems.drop(['playtime_2weeks','user_url','item_name','items_count','playtime_forever'], axis=1)

In [ ]:
df_usersItems.columns

Index(['item_id', 'user_id', 'hours_game'], dtype='object')

In [ ]:
#Borramos dulplicados
df_usersItems = df_usersItems.drop_duplicates(keep='first')

In [ ]:
df_usersItems

,item_id,user_id,hours_game
0,10,76561197970982479,0.10
2,30,76561197970982479,0.12
8,300,76561197970982479,78.88
9,240,76561197970982479,30.88
10,3830,76561197970982479,5.55
...,...,...,...
5153202,304930,76561198329548331,11.28
5153203,227940,76561198329548331,0.72
5153206,388490,76561198329548331,0.05
5153207,521570,76561198329548331,0.07


# 4. Almacenar el conjunto de datos procesado
Igual que el anterior, este tambien en PARRQUET en la carpeta especificada.

In [ ]:
df_usersItems.to_parquet('C:\\Users\\Juampi\\Desktop\\Henry\\PI_MLOps\\Datasets\\archivos_procesados\\users_items_cleaned.parquet')

Repitamos este proceso para el ultimo archivo: australian_user_reviews.json.

# 1. Descargar el conjunto de datos:

**Fuente de datos: australian_user_reviews.json**

⚠️Nota: Este comando descarga el modelo de análisis de sentimiento de NLTK y solo debe ejecutarse la primera vez que se utiliza.

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Juampi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Es importante destacar que el archivo inicial presentaba inconsistencias en su formato JSON, lo que llevó a su corrupción. Para resolver este problema, utilicé el editor de JSON de VSC para reparar el archivo.

In [3]:
#id del archivo en Google Drive y nombre del archivo local
Id = '1XwAffkpCpj4DhG1-NAGOm1iyiZB5N7DL'
output = 'australian_user_reviews.json'

#Utilizar la función (descargar_leer_json) para descargar y leer el JSON
df_UserReviews = descargar_leer_json(Id, output)

In [4]:
reviews_respaldo= df_UserReviews

# 2. Análisis e interpretación de los datos

In [5]:
#Veamos un poco los datos
df_UserReviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Este dataframe consta de tres columnas principales: 'user_id', 'user_url' y 'reviews'. Aquí hay algunas observaciones sobre cada una:

* La columna 'user_id'parece contener identificadores únicos para cada usuario, lo que sugiere que es un campo clave para identificar a los usuarios en el conjunto de datos.

* 'user_url' Contiene enlaces a los perfiles de los usuarios en la plataforma Steam. Tiene poca reelevancia para este proyecto.

* La mas intersante: 'reviews'. Esta columna parece contener información detallada sobre las revisiones realizadas por cada usuario. Cada entrada en esta columna es un diccionario o estructura similar, que incluye detalles como la fecha de publicación, el contenido de la revisión, etc.

Es evidente que la columna 'reviews' contiene datos estructurados o anidados en formato JSON. Desanidaremos estos datos como lo hicimos con 'items' del archivo anterior.

In [6]:
# Desanidamos la columna review
desanidado = []
for index, row in df_UserReviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    for review_dict in row['reviews']:
        review_dict['user_id'] = user_id
        review_dict['user_url'] = user_url
        desanidado.append(review_dict)

df_UserReviews = pd.DataFrame(desanidado)

In [7]:
#Veamos como quedo nuestro Dataframe
df_UserReviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637


In [8]:
#Veamos como esta estructurado
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        59305 non-null  object
 1   posted       59305 non-null  object
 2   last_edited  59305 non-null  object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  bool  
 6   review       59305 non-null  object
 7   user_id      59305 non-null  object
 8   user_url     59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB



Dado que las columnas principalmente contienen tipos de datos 'object' y 'bool', aplicar estadísticas descriptivas a todas las columnas podría no ser tan significativo. Sin embargo, en relación con la columna 'recommend', es importante observar sus valores para obtener una perspectiva clara de la distribución de recomendaciones en el conjunto de datos.

In [9]:
# Calculamos valores en la columna 'recommend' 
conteo_por_recomendaciones = df_UserReviews['recommend'].value_counts()
porcentaje_recomendaciones = df_UserReviews['recommend'].value_counts(normalize=True) * 100
#porcentaje_recomendaciones = porcentaje_recomendaciones.round(2).astype(str) + '%'

# Creamos un nuevo DataFrame con el conteo y porcentaje
resumen_recomendaciones = pd.DataFrame({
    'Conteo': conteo_por_recomendaciones,
    'Porcentaje': porcentaje_recomendaciones.round(2).astype(str) + '%'
})

# Ordenamos el DataFrame por el conteo de mayor a menor
resumen_recomendaciones = resumen_recomendaciones.sort_values(by='Conteo', ascending=False)
print(resumen_recomendaciones)

           Conteo Porcentaje
recommend                   
True        52473     88.48%
False        6832     11.52%


# 3. Limpieza y procesamiento

Entre ortas cosas, realizaremos análisis de texto para determinar el idioma de las reseñas con langdetect y realizaremos análisis de sentimiento para categorizar las reseñas con nltk.

In [10]:
# Reemplazar valores vacíos, 'null' y 'None' con NaN en todo el DataFrame
df_UserReviews.replace(['', 'null', 'None'], np.nan, inplace=True)

In [11]:
# Filas donde todas las columnas especificadas tienen valores nulos.
columnas_a_considerar = ['posted', 'recommend', 'review']
filas_con_nulos = df_UserReviews[df_UserReviews[columnas_a_considerar].isnull().all(axis=1)]
filas_con_nulos

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url


Las 'columnas_a_considerar' representan aspectos críticos de las revisiones de usuarios, y la ausencia simultánea de datos en estas áreas podría afectar negativamente la calidad de nuestros resultados.

Afortunadamente, los resultados muestran que dentro del conjunto de datos actual, no hay filas donde 'posted', 'recommend' y 'review' estén simultáneamente ausentes. Esto significa que tenemos información disponible en todas las áreas críticas para nuestro análisis. Por lo tanto, podemos proceder con confianza en nuestra exploración y análisis de los datos.

In [12]:
# Se establece umbral del 80% para decidir que columnas eliminar por valores nulos
umbral_nulos = 0.8

# Calcula el porcentaje de valores nulos por columna
porcentaje_nulos = df_UserReviews.isnull().mean()

# Filtra las columnas que superan el umbral
columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > umbral_nulos]

# Muestra las columnas y su respectivo porcentaje de valores nulos
print("Columnas con más del {}% de valores nulos (candidatas a eliminar):".format(umbral_nulos * 100))
for columna, porcentaje in columnas_a_eliminar.items():
    print("{}: {:.2%}".format(columna, porcentaje))

Columnas con más del 80.0% de valores nulos (candidatas a eliminar):
funny: 86.26%
last_edited: 89.65%



La mayoría de las filas en las columnas 'funny' y 'last_edited' tienen valores nulos, y dado que no son cruciales para nuestro análisis, podemos descartarlas sin perder información significativa. Además, las columnas 'helpful' y 'user_url' tampoco aportan datos relevantes para nuestro estudio objetivo. Por lo tanto, podemos proceder eliminando estas columnas del conjunto de datos para simplificar nuestro análisis y reducir el ruido en los datos.

In [13]:
# Eliminamos las columnas que pasan el umbral establecido para valores nulos 
df_UserReviews.drop(columns=columnas_a_eliminar.index, inplace=True)

# Se eliminan las columnas 'helpful' y 'user_url' por considerarse no relevantes
df_UserReviews.drop(['helpful','user_url'], axis=1, inplace=True)

In [14]:
# Se buscan registros duplicados
df_UserReviews.sort_values('user_id')
filas_duplicadas = df_UserReviews[df_UserReviews.duplicated(subset=['user_id', 'item_id', 'posted', 'review'], keep=False)]
filas_duplicadas.count()

posted       1736
item_id      1736
recommend    1736
review       1736
user_id      1736
dtype: int64

In [15]:
# Se eliminan los registros duplicados basándome en múltiples columnas 
duplicados_eliminados = df_UserReviews.drop_duplicates(subset=['user_id', 'item_id', 'posted', 'review'], keep=False, inplace=True)

cantidad_total_duplicados_eliminados = filas_duplicadas.shape[0]

print("Cantidad total de registros duplicados eliminados:", cantidad_total_duplicados_eliminados)

Cantidad total de registros duplicados eliminados: 1736


Dentro del análisis de los reviews, es crucial conocer el año en que fueron realizados. Aunque en mi conjunto de datos no contamos con una columna explícita de año, disponemos de la información en la columna 'posted'.

En consecuencia, se ha decidido crear una nueva columna llamada 'year', de tipo entero, extrayendo la información de la columna 'posted'. Posteriormente, la columna 'posted' se eliminará para simplificar la estructura del conjunto de datos. Este proceso asegura que tengamos un atributo específico para el año de cada review, facilitando análisis temporales y tendencias a lo largo del tiempo.

In [16]:
# Asegúrate de que 'posted' sea de tipo datetime
df_UserReviews['posted'] = pd.to_datetime(df_UserReviews['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

# Crea la columna 'year' a partir de 'posted'
df_UserReviews['year'] = df_UserReviews['posted'].dt.year.astype('Int64')

# Ordena el DataFrame por 'item_id' y 'year' para asegurar que la interpolación se haga correctamente
df_UserReviews = df_UserReviews.sort_values(['item_id', 'year'])

# Rellenar valores nulos en 'year' mediante interpolación lineal por grupo (item_id)
df_UserReviews['year'] = df_UserReviews.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)

# Si aún hay valores nulos después de la interpolación, se llenan con la mediana. 
df_UserReviews['year'] = df_UserReviews['year'].fillna(df_UserReviews['year'].median())

In [17]:
# Se eliminan las columnas 'posted' y 'user_id
df_UserReviews.drop(['posted','user_id'], axis=1, inplace=True)
df_UserReviews.head()

,item_id,recommend,review,year
5331,10,True,this game is the 1# online action game is awes...,2011
22702,10,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !!...,2012
35539,10,True,:D,2012
43134,10,True,Good Game :D,2012
24137,10,True,jueguenlooooooo,2013


Identificamos que la columna 'posted' contenía algunos valores estaban ausentes en cuanto al año. Dado que la temporalidad es esencial para nuestro análisis, decidimos abordar este problema de manera estratégica una vez creada la columna 'year'.

* Selección de Técnica de Imputación: Optamos por utilizar la técnica de interpolación para llenar los valores faltantes en 'year'. La interpolación es útil cuando existe una relación secuencial o temporal en los datos.

* Agrupación por Juego ('item_id'): Dado que cada review está asociado a un juego único, agrupamos los datos por 'item_id' para considerar la relación temporal dentro de cada juego.

* Aplicación de Interpolación Lineal por Grupo: Aplicamos la interpolación lineal a cada grupo de 'item_id', lo que permitió estimar los años faltantes basándonos en los años conocidos del mismo juego. Se consideró la opción de aplicar la interpolación lineal global, pero optamos por la interpolación por grupo para capturar posibles variaciones en la temporalidad entre diferentes juegos.

* Manejo de Valores Nulos Restantes: Después de la interpolación, si aún había valores nulos, llenamos esos espacios con la mediana de la columna 'year'. Este proceso asegura que nuestro conjunto de datos mantenga la coherencia temporal necesaria para análisis posteriores, y la elección de la interpolación por grupo se alinea con la naturaleza de los datos, donde la temporalidad puede variar entre diferentes juegos. NOTA: Para una 2da versión los registos nulos restantes se podrían llenar con inicialmente con el año de publicación del juego 'release_date' en el archivo steam_games.json y por último con la mediana.

Ahora, evaluaremos la columna 'review' para obtener estadísticas sobre la cantidad de reseñas por lenguaje. Este análisis será crucial para determinar si el proceso de análisis de sentimientos se realizará en todo el conjunto de datos o si se aplicará un filtro específico por idioma.

In [18]:
# Análisis de texto para determinar el idioma de las reseñas

def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return None

# Aplicar la función para detectar idioma y crear una nueva columna 'language'
df_UserReviews['language'] = df_UserReviews['review'].apply(detectar_idioma)

# Calcular el conteo y porcentaje de cada idioma
conteo_por_idioma = df_UserReviews['language'].value_counts()
porcentaje_por_idioma = df_UserReviews['language'].value_counts(normalize=True) * 100

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_idiomas = resumen_idiomas.sort_values(by='Conteo', ascending=False)
resumen_idiomas.head()

,Conteo,Porcentaje
language,,
en,44426,77.92%
pt,2115,3.71%
es,1249,2.19%
de,1115,1.96%
so,997,1.75%


Con los resultados obtenidos consideré realizar el análisis de sentimiento solo en los registros en inglés.

In [19]:
#esto es para tener la información en el EDA

# Mapeo de códigos de idioma a nombres completos (Top 5)
mapeo_idiomas = {
    'en': 'English',
    'pt': 'Portugués',
    'es': 'Español',
    'de': 'German',
    'so': 'Somali',
    # Agrega más mapeos según sea necesario
}

# Aplicar el mapeo al DataFrame
df_UserReviews['idioma_completo'] = df_UserReviews['language'].map(mapeo_idiomas)

# Crear un DataFrame con el resumen de idiomas
resumen_idiomas = df_UserReviews['idioma_completo'].value_counts().reset_index()
resumen_idiomas.columns = ['Idioma', 'Conteo']
resumen_idiomas['Conteo'] = resumen_idiomas['Conteo'].round(2)

# Calcular el porcentaje
resumen_idiomas['Porcentaje'] = (resumen_idiomas['Conteo'] / len(df_UserReviews)) * 100
resumen_idiomas['Porcentaje'] = resumen_idiomas['Porcentaje'].round(2)

# Guardar en un archivo CSV
resumen_idiomas.to_parquet('C:\\Users\\Juampi\\Desktop\\Henry\\PI_MLOps\\Datasets\\archivos_procesados\\resumen_idiomas.parquet')

In [20]:
# Filtrar los registros donde 'language' no es igual a 'en' 
df_UserReviews = df_UserReviews[df_UserReviews['language'] == 'en']

# Feature Engineering

La columna 'review' también será parte de nuestro estudio ya que incluye reseñas de juegos hechos por distintos usuarios. Partiendo de 'review' se va crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: Debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. De no ser posible este análisis por estar ausente la reseña escrita, tomará el valor de 1. Esta nueva columna reemplaza la de 'review' para facilitar el trabajo de los modelos de machine learning y el análisis de datos.

In [21]:
#Se convierten todas las letras a minúsculas para asegurar que todas las palabras sean tratadas de la misma manera.
df_UserReviews.loc[:, 'review'] = df_UserReviews['review'].str.lower()

In [22]:
#Eliminación de caracteres especiales
df_UserReviews['review'] = df_UserReviews['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)

In [23]:
#Elimina caracteres de puntuación que no aportan al análisis de sentimiento.
df_UserReviews.loc[:, 'review'] = df_UserReviews['review'].str.replace('[^\w\s]', '', regex=True)

In [24]:
# Análisis de sentimiento para categorizar las reseñas columna 'review' 

def analyze_sentiments(df):
    # Instanciar el analizador de sentimientos
    sia = SentimentIntensityAnalyzer()

    # Aplicar el análisis de sentimientos y asignar valores numéricos
    df['compound_score'] = df['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
    df['sentiment_analysis'] = df['compound_score'].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    score_counts = df['sentiment_analysis'].value_counts()

    # Conteo de reviews en blanco
    blank_reviews_count = df['review'].isnull().sum()

    # Total de reviews
    total_reviews = len(df)

    # Calcular porcentajes
    score_percentages = (score_counts / total_reviews * 100).round(2)
    blank_reviews_percentage = (blank_reviews_count / total_reviews * 100).round(2)
     
    # Se eliminan las columnas 'review' y 'compound_score', no necesitaremos estos datos
    df.drop(['review','compound_score'], axis=1, inplace=True)  

    return df, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage

In [25]:
# Llamar a la función analyze_sentiments
df_UserReviews, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage = analyze_sentiments(df_UserReviews)

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_sentimientos = pd.DataFrame({
    'Conteo': score_counts,
    'Porcentaje': score_percentages.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_sentimientos = resumen_sentimientos.sort_values(by='Conteo', ascending=False)

# Imprimir los resultados
print("\nResumen de análisis de sentimientos:")
print(resumen_sentimientos)
print("\nConteo de reviews en blanco: ", blank_reviews_count, " Porcentaje: ", blank_reviews_percentage.round(2).astype(str) + '%')


Resumen de análisis de sentimientos:
                    Conteo Porcentaje
sentiment_analysis                   
2                    31453      70.8%
0                     8283     18.64%
1                     4690     10.56%

Conteo de reviews en blanco:  0  Porcentaje:  0.0%


In [26]:
df_UserReviews

,item_id,recommend,year,language,idioma_completo,sentiment_analysis
5331,10,True,2011,en,English,2
45506,10,True,2013,en,English,1
7801,10,True,2014,en,English,2
7967,10,True,2014,en,English,2
8519,10,True,2014,en,English,2
...,...,...,...,...,...,...
51725,99900,True,2015,en,English,2
53065,99900,False,2015,en,English,2
12393,99910,True,2011,en,English,2
53052,99910,False,2014,en,English,0


# 4. Almacenar el conjunto de datos procesado

In [27]:
df_UserReviews.to_parquet('C:\\Users\\Juampi\\Desktop\\Henry\\PI_MLOps\\Datasets\\archivos_procesados\\user_reviews_cleaned.parquet')